In [1]:
%load_ext rustdef

load rustdef


<IPython.core.display.Javascript object>

     Created library package
    Updating crates.io index
      Adding pyo3 v0.15.1 to dependencies.
             Features:
             + extension-module
             + indoc
             + macros
             + paste
             + pyo3-macros
             + unindent
             - abi3
             - abi3-py310
             - abi3-py36
             - abi3-py37
             - abi3-py38
             - abi3-py39
             - anyhow
             - auto-initialize
             - eyre
             - hashbrown
             - indexmap
             - inventory
             - multiple-pymethods
             - nightly
             - num-bigint
             - num-complex
             - serde


In [2]:
%rustdef deps add --help

usage: %rustdef deps add [-h] args [args ...]

Add dependencies

positional arguments:
  args        Dependencies to be added. See cargo-edit

options:
  -h, --help  show this help message and exit


In [3]:
%rustdef deps add ndarray@0.15.0 numpy@0.15.0

    Updating crates.io index
      Adding ndarray v0.15.0 to dependencies.
             Features:
             + std
             - approx
             - blas
             - cblas-sys
             - docs
             - libc
             - matrixmultiply-threading
             - rayon
             - rayon_
             - serde
             - serde-1
             - test
      Adding numpy v0.15.0 to dependencies.
             Features:
             - rayon


In [4]:
%rustdef deps show

ndarray = "0.15.0"
numpy = "0.15.0"

[pyo3]
version = "0.15.1"
features = [ "extension-module",]



In [5]:
%%rustdef
use numpy::{IntoPyArray, PyArrayDyn, PyReadonlyArrayDyn};
use ndarray::{ArrayD, ArrayViewD};

// immutable example
fn axpy_rs(a: f64, x: ArrayViewD<'_, f64>, y: ArrayViewD<'_, f64>) -> ArrayD<f64> {
    a * &x + &y
}

#[pyfn(m, "axpy")]
fn axpy<'py>(
    py: Python<'py>,
    a: f64,
    x: PyReadonlyArrayDyn<'py, f64>,
    y: PyReadonlyArrayDyn<'py, f64>,
) -> &'py PyArrayDyn<f64> {
    let x = x.as_array();
    let y = y.as_array();
    axpy_rs(a, x, y).into_pyarray(py)
}

Building..
🔗 Found pyo3 bindings
🐍 Found CPython 3.10 at /usr/local/anaconda3/envs/rustdef-py311/bin/python
   Compiling autocfg v1.1.0
   Compiling proc-macro2 v1.0.47
   Compiling quote v1.0.21
   Compiling unicode-ident v1.0.5
   Compiling pyo3-build-config v0.15.2
   Compiling syn v1.0.105
   Compiling proc-macro-hack v0.5.19
   Compiling once_cell v1.16.0
   Compiling libc v0.2.138
   Compiling cfg-if v1.0.0
   Compiling parking_lot_core v0.8.5
   Compiling smallvec v1.10.0
   Compiling scopeguard v1.1.0
   Compiling unindent v0.1.10
   Compiling rawpointer v0.2.1
   Compiling cfg-if v0.1.10
   Compiling instant v0.1.12              ] 0/55: quote(build.rs), parking_...
   Compiling matrixmultiply v0.3.2        ] 1/55: quote(build.rs), parking_...
   Compiling num-traits v0.2.15          ] 16/55: libc(build), proc-macro-h...
   Compiling lock_api v0.4.9
   Compiling num-integer v0.1.45
   Compiling paste-impl v0.1.18          ] 28/55: num-traits(build), parkin...    Building [=====

In [6]:
import numpy as np

In [7]:
axpy(2.4, np.array([1.0, 3.2, 2.9]), np.array([2.3, 10.9, -2]))

array([ 4.7 , 18.58,  4.96])

In [8]:
%%rustdef
use numpy::{IntoPyArray, PyArrayDyn, PyReadonlyArrayDyn};
use ndarray::{ArrayD, ArrayViewD};

// mutable example
#[pyfn(m, "mult_inline")]
fn mult_inline<'py>(_py: Python<'py>, a: f64, x: &'py PyArrayDyn<f64>) -> PyResult<()> {
    let mut x = unsafe { x.as_array_mut() };
    x *= a;
    Ok(())
}

Building..
🔗 Found pyo3 bindings
🐍 Found CPython 3.10 at /usr/local/anaconda3/envs/rustdef-py311/bin/python
   Compiling rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099 v0.1.0 (/Users/ryosuke.kamesawa.mot/.rustdef/rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099)
  --> rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099/src/lib.rs:12:11
   |
12 | #[pyfn(m, "mult_inline")]
   |           ^^^^^^^^^^^^^
   |
   = note: `#[warn(deprecated)]` on by default


    Building warning: 1 warning emitted

[=========================> ] 54/55: rustdef_cell_eb290401c4e6...
    Finished dev [unoptimized + debuginfo] target(s) in 1.30sl_eb290401c4e6...
📦 Built wheel for CPython 3.10 to /Users/ryosuke.kamesawa.mot/.rustdef/target/wheels/rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099-0.1.0-cp310-cp310-macosx_10_7_x86_64.whl
Processing /Users/ryosuke.kamesawa.mot/.rustdef/target/wheels/rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099-0.1.0-cp310-cp310-macosx_10_7_x86_64.whl


In [9]:
x = np.random.rand(10)
y = np.random.rand(10)
x, y

(array([0.03543514, 0.57331017, 0.3636721 , 0.69263841, 0.14796075,
        0.94630033, 0.29753375, 0.04364509, 0.78102218, 0.77589358]),
 array([0.21238349, 0.79239126, 0.79260236, 0.27942979, 0.15694099,
        0.99800985, 0.57824666, 0.06794508, 0.77480404, 0.48441381]))

In [10]:
axpy(3.0, x, y)

array([0.31868893, 2.51232176, 1.88361866, 2.35734502, 0.60082323,
       3.83691084, 1.47084791, 0.19888035, 3.11787057, 2.81209456])

In [11]:
mult_inline(12.0, x)

In [12]:
x

array([ 0.42522172,  6.87972199,  4.36406519,  8.31166095,  1.77552899,
       11.35560397,  3.57040501,  0.52374106,  9.37226613,  9.31072298])